# Session 1.3: Prompt Engineering and Templates

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1h5mUoP7Sgit2ZW6bWRQV5qiDWpiBnRj1?usp=sharing)

## Overview

Prompt engineering is the art of designing effective instructions for LLMs. In this notebook, you'll learn:

- **Prompt basics** and best practices
- **Prompt templates** in LangChain
- **Few-shot prompting**
- **Output parsers** for structured responses
- **Prompt composition** and chaining

### Learning Objectives

✅ Master prompt engineering techniques  
✅ Create reusable prompt templates  
✅ Use few-shot examples effectively  
✅ Parse and structure LLM outputs  
✅ Compose complex prompts  

In [ ]:
# Install required packages
!pip install -q langchain langchain-openai langchain-core langchain-community
!pip install -q python-dotenv pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
from langchain_openai import ChatOpenAI

In [ ]:
import os
from google.colab import userdata

# Set OpenAI API key from Google Colab's user environment or default
def set_openai_api_key(default_key: str = "YOUR_API_KEY") -> None:
    """Set the OpenAI API key from Google Colab's user environment or use a default value."""
    #if not (userdata.get("OPENAI_API_KEY") or "OPENAI_API_KEY" in os.environ):
    try:
      os.environ["OPENAI_API_KEY"] = userdata.get("MDX_OPENAI_API_KEY")
    except:
      os.environ["OPENAI_API_KEY"] = default_key

set_openai_api_key()

# Initialize LLM
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)
llm = ChatOpenAI(model="gpt-5-nano")

## 1. Prompt Engineering Fundamentals

### Key Principles

1. **Be Specific**: Clear, detailed instructions
2. **Provide Context**: Give background information
3. **Use Examples**: Show desired format (few-shot)
4. **Set Constraints**: Define length, format, tone
5. **Iterate**: Test and refine prompts

In [ ]:
# Bad vs Good Prompts

# ❌ Bad: Too vague
bad_prompt = "Tell me about AI"

# ✅ Good: Specific and structured
good_prompt = """
Explain artificial intelligence to a 10-year-old.
Include:
- A simple definition
- One real-world example
- Why it's useful
Keep your response under 100 words.
"""

print("Bad Prompt Response:")
print(llm.invoke(bad_prompt).content)
print("\n" + "="*50 + "\n")
print("Good Prompt Response:")
print(llm.invoke(good_prompt).content)

Bad Prompt Response:
AI, or artificial intelligence, is a field in computer science focused on building systems that can perform tasks that usually require human intelligence. These tasks include understanding language, recognizing images, solving problems, learning from data, and making decisions.

Key ideas and distinctions
- Narrow AI vs. general AI: Narrow (or weak) AI is designed for specific tasks (e.g., voice assistants, image classifiers, recommendation systems). General AI would be capable of any intellectual task a human can do, and is not yet reachable. Superintelligent AI, a hypothetical future stage, would surpass human intelligence across almost all areas.
- Data-driven learning: Modern AI mostly uses data and statistical methods to learn patterns. The model improves by optimizing a mathematical objective (loss function) with lots of examples.
- Difference from traditional software: Traditional software follows explicit rules written by humans. AI, especially machine lear

## 2. Introduction to Prompt Templates

**Prompt Templates** allow you to create reusable prompts with variables.

In [ ]:
from langchain_core.prompts import PromptTemplate

# Create a simple template
template = PromptTemplate(
    input_variables=["topic", "audience"],
    template="Explain {topic} to a {audience}. Keep it simple and engaging."
)

# Use the template
prompt1 = template.format(topic="blockchain", audience="teenager")
prompt2 = template.format(topic="quantum computing", audience="business executive")

print("Prompt 1:", prompt1)
print("\nPrompt 2:", prompt2)

Prompt 1: Explain blockchain to a teenager. Keep it simple and engaging.

Prompt 2: Explain quantum computing to a business executive. Keep it simple and engaging.


## 3. Chat Prompt Templates

For chat models, use **ChatPromptTemplate** to structure system, human, and AI messages.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# Create a chat prompt template
chat_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful {role} who explains concepts clearly."),
    ("human", "Explain {concept} in {num_sentences} sentences.")
])

# Format the prompt
messages = chat_template.format_messages(
    role="data scientist",
    concept="neural networks",
    num_sentences="3"
)

print("Messages:")
for msg in messages:
    print(f"{msg.type}: {msg.content}\n")

# Invoke LLM with template
response = llm.invoke(messages)
print("Response:")
print(response.content)

Messages:
system: You are a helpful data scientist who explains concepts clearly.

human: Explain neural networks in 3 sentences.

Response:
A neural network is a collection of interconnected units (neurons) arranged in layers that transform input data through weighted sums and nonlinear activation functions. Each connection has a weight that the model adjusts during training, allowing it to learn complex patterns from examples. Training uses algorithms like backpropagation with gradient descent to minimize a loss function, updating weights to approximate the desired input–output mapping.


## 4. Using LCEL (LangChain Expression Language)

LCEL provides a clean syntax for chaining components.

In [ ]:
# Create a chain using the | (pipe) operator
chain = chat_template | llm

# Invoke the chain
response = chain.invoke({
    "role": "teacher",
    "concept": "photosynthesis",
    "num_sentences": "2"
})

print(response.content)

Photosynthesis is the process by which green plants, algae, and some bacteria capture light energy to convert water and carbon dioxide into glucose and oxygen. The light-dependent reactions use light to produce ATP and NADPH, which power the Calvin cycle to synthesize glucose from CO2.


## 5. Few-Shot Prompting

Provide examples to guide the model's responses.

In [ ]:
from langchain_core.prompts import FewShotPromptTemplate

# Define examples
examples = [
    {
        "question": "What is Python?",
        "answer": "Python is a high-level, interpreted programming language known for its simplicity and readability."
    },
    {
        "question": "What is JavaScript?",
        "answer": "JavaScript is a versatile programming language primarily used for creating interactive web pages."
    }
]

# Create example template
example_template = PromptTemplate(
    input_variables=["question", "answer"],
    template="Question: {question}\nAnswer: {answer}"
)

# Create few-shot template
few_shot_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_template,
    prefix="Answer the following questions in a clear, technical manner:\n",
    suffix="Question: {input}\nAnswer:",
    input_variables=["input"]
)

# Test the few-shot prompt
prompt = few_shot_template.format(input="What is Rust?")
print("Few-Shot Prompt:")
print(prompt)
print("\n" + "="*50 + "\n")

response = llm.invoke(prompt)
print("Response:")
print(response.content)

Few-Shot Prompt:
Answer the following questions in a clear, technical manner:


Question: What is Python?
Answer: Python is a high-level, interpreted programming language known for its simplicity and readability.

Question: What is JavaScript?
Answer: JavaScript is a versatile programming language primarily used for creating interactive web pages.

Question: What is Rust?
Answer:


Response:
Question: What is Rust?
Answer: Rust is a systems programming language that emphasizes performance, safety, and concurrency. It compiles to native code and uses a strict ownership and borrowing model to enforce memory safety without a garbage collector, enabling predictable performance. It offers zero-cost abstractions, a strong static type system, and modern tooling (Cargo, crates.io), making it suitable for low-level, performance-critical, and concurrent applications as well as embedded systems.


## 6. Output Parsers

Parse LLM outputs into structured formats.

In [ ]:
from langchain_core.output_parsers import StrOutputParser, CommaSeparatedListOutputParser

# String Output Parser (default)
str_parser = StrOutputParser()

# Create a chain with output parser
template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", "{input}")
])

chain = template | llm | str_parser

result = chain.invoke({"input": "Say hello!"})
print(f"Type: {type(result)}")
print(f"Result: {result}")

Type: <class 'str'>
Result: Hello! 👋 How can I help you today?


In [ ]:
# Comma-Separated List Parser
list_parser = CommaSeparatedListOutputParser()

list_template = ChatPromptTemplate.from_messages([
    ("system", "Generate a comma-separated list as requested."),
    ("human", "{input}\n{format_instructions}")
])

# Create chain with list parser
list_chain = list_template | llm | list_parser

result = list_chain.invoke({
    "input": "List 5 programming languages",
    "format_instructions": list_parser.get_format_instructions()
})

print(f"Type: {type(result)}")
print(f"Result: {result}")

Type: <class 'list'>
Result: ['Python', 'JavaScript', 'Java', 'C++', 'Ruby']


## 7. Structured Output with Pydantic

Use Pydantic models for type-safe, validated outputs.

In [ ]:
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field

# Define a Pydantic model
class Person(BaseModel):
    name: str = Field(description="The person's name")
    age: int = Field(description="The person's age")
    occupation: str = Field(description="The person's job")

# Create JSON parser
parser = JsonOutputParser(pydantic_object=Person)

# Create prompt with format instructions
template = ChatPromptTemplate.from_messages([
    ("system", "Generate fictional person data as JSON."),
    ("human", "{query}\n{format_instructions}")
])

chain = template | llm | parser

result = chain.invoke({
    "query": "Generate data for a data scientist",
    "format_instructions": parser.get_format_instructions()
})

print(f"Type: {type(result)}")
print(f"Result: {result}")
print(f"Name: {result['name']}")
print(f"Age: {result['age']}")

Type: <class 'dict'>
Result: {'name': 'Alex Rivera', 'age': 29, 'occupation': 'Data Scientist'}
Name: Alex Rivera
Age: 29


## 8. Partial Templates

Pre-fill some variables while leaving others dynamic.

In [ ]:
from datetime import datetime

def get_current_date():
    return datetime.now().strftime("%Y-%m-%d")

# Create template with partial variables
template = PromptTemplate(
    input_variables=["topic"],
    template="Today's date is {date}. Write a brief summary about {topic}.",
    partial_variables={"date": get_current_date}
)

# Use the template
prompt = template.format(topic="quantum computing")
print(prompt)
print("\nResponse:")
print(llm.invoke(prompt).content)

Today's date is 2025-10-23. Write a brief summary about quantum computing.

Response:
Here’s a brief overview of quantum computing:

- Core idea: Quantum computers use qubits that can be in superposition and become entangled, enabling computation with quantum gates that manipulate these states.
- How it works: Quantum circuits evolve qubits with unitary operations and are read out probabilistically; interference helps amplify correct results.
- Why it matters: For some problems, quantum algorithms can offer substantial speedups—for example, Shor’s algorithm for factoring and Grover’s search—though not for every task.
- Current stage: We’re in the Noisy Intermediate-Scale Quantum (NISQ) era—devices with tens to a few hundred qubits that are noisy and not yet fault-tolerant.
- Hardware approaches: Leading platforms include superconducting qubits, trapped ions, and photonic qubits; several companies and research groups are advancing each path.
- Near-term applications: Chemistry and mater

## 9. Prompt Composition

Combine multiple prompts into complex workflows.

In [ ]:
from langchain_core.prompts import PipelinePromptTemplate

# Define sub-prompts
intro_template = PromptTemplate(
    input_variables=["topic"],
    template="Introduction to {topic}:"
)

body_template = PromptTemplate(
    input_variables=["introduction", "details"],
    template="{introduction}\n\n{details}"
)

# Compose prompts
full_template = PromptTemplate(
    input_variables=["topic", "details"],
    template="{topic_intro}\n\nKey Details:\n{details}"
)

# Use composition
intro = intro_template.format(topic="Machine Learning")
final_prompt = body_template.format(
    introduction=intro,
    details="- Supervised learning\n- Unsupervised learning\n- Reinforcement learning"
)

print(final_prompt)

Introduction to Machine Learning:

- Supervised learning
- Unsupervised learning
- Reinforcement learning


## 10. Advanced: Chain-of-Thought Prompting

Guide the model to break down complex problems step-by-step.

In [ ]:
cot_template = ChatPromptTemplate.from_messages([
    ("system", "You are a logical problem solver. Break down problems step-by-step."),
    ("human", """
Problem: {problem}

Let's solve this step by step:
1. First, identify what we know
2. Then, determine what we need to find
3. Finally, solve and verify

Please provide your reasoning:""")
])

cot_chain = cot_template | llm | StrOutputParser()

result = cot_chain.invoke({
    "problem": "If a train travels at 60 mph for 2.5 hours, how far does it travel?"
})

print(result)

Here is a concise step-by-step solution:

- Step 1: Identify what is given: speed = 60 mph, time = 2.5 hours.
- Step 2: Use the formula: distance = speed × time.
- Step 3: Compute: 60 × 2.5 = 150.
- Step 4: State the result with units: distance = 150 miles.
- Quick check: mph × hours works out to miles, so the units are correct.

Answer: 150 miles.


## 11. Prompt Best Practices Summary

### ✅ DO:
- Be specific and clear
- Provide context and examples
- Use consistent formatting
- Test with edge cases
- Version control your prompts

### ❌ DON'T:
- Use ambiguous language
- Mix multiple tasks in one prompt
- Assume implicit knowledge
- Forget to set constraints
- Hard-code values (use templates)

## 🎯 Exercise 4: Build a Code Explainer

**Task**: Create a prompt template that:
1. Takes a code snippet and programming language
2. Explains what the code does
3. Identifies key concepts
4. Returns structured output (JSON)

In [ ]:
# Define your Pydantic model
class CodeExplanation(BaseModel):
    summary: str = Field(description="One-sentence summary")
    explanation: str = Field(description="Detailed explanation")
    concepts: list = Field(description="List of key concepts")
    complexity: str = Field(description="Beginner/Intermediate/Advanced")

# TODO: Implement your code explainer chain
def explain_code(code_snippet, language):
    """
    Explain a code snippet using LLM

    Args:
        code_snippet: The code to explain
        language: Programming language

    Returns:
        Structured explanation
    """
    pass

# Test with example
# code = """
# def fibonacci(n):
#     if n <= 1:
#         return n
#     return fibonacci(n-1) + fibonacci(n-2)
# """
# explain_code(code, "Python")

## 🎯 Exercise 5: Dynamic Few-Shot Learning

**Task**: Create a system that:
1. Selects relevant examples based on the query
2. Uses semantic similarity to find best examples
3. Dynamically constructs few-shot prompts

In [ ]:
from langchain_openai import OpenAIEmbeddings
from numpy import dot
from numpy.linalg import norm

def dynamic_few_shot(query, example_pool, k=2):
    """
    Select most relevant examples using embeddings

    Args:
        query: User's query
        example_pool: List of example dicts with 'input' and 'output'
        k: Number of examples to select

    Returns:
        Response using dynamically selected examples
    """
    # TODO: Implement dynamic example selection
    pass

# Test data
# examples = [
#     {"input": "Translate 'hello' to French", "output": "bonjour"},
#     {"input": "Translate 'goodbye' to French", "output": "au revoir"},
#     {"input": "What is 2+2?", "output": "4"},
#     {"input": "What is 5*5?", "output": "25"}
# ]
# dynamic_few_shot("Translate 'thank you' to French", examples)

## Summary

In this notebook, you learned:

✅ Prompt engineering fundamentals  
✅ Creating and using prompt templates  
✅ Chat prompt templates and LCEL  
✅ Few-shot prompting techniques  
✅ Output parsers for structured data  
✅ Pydantic models for type safety  
✅ Prompt composition strategies  
✅ Chain-of-thought reasoning  

**Next**: We'll explore Memory and Conversation Management!